# Viterbi Algorithm Based Phoneme Decoding for Speech Recognition

## Aim
To implement the Viterbi Algorithm in a Hidden Markov Model (HMM) and decode the most probable phoneme sequence for the word "hello".

## Problem Description

Speech recognition can be modeled as a sequence problem where we want to find the most likely sequence of hidden states (phonemes) given a sequence of observations (acoustic features).

In this experiment:
- **Hidden States (S)**: Represent phonemes. We have 4 states: $S_1=/h/, S_2=/e/, S_3=/l/, S_4=/o/$.
- **Observations (O)**: Represent extracted acoustic feature vectors: $O_1, O_2, O_3, O_4$.
- **Goal**: Find the state sequence $Q = q_1, q_2, q_3, q_4$ that maximizes $P(Q|O, \lambda)$, where $\lambda$ is the HMM model parameters $(A, B, \pi)$.

The **Viterbi Algorithm** is a dynamic programming algorithm used to find this most likely sequence of hidden states, often called the "Viterbi path". It avoids the exponential complexity of checking all possible paths by keeping track of the maximum probability path reaching each state at each time step.

### HMM Diagram

```mermaid
graph LR
    h((/h/)) -->|1.0| e((/e/))
    e -->|1.0| l((/l/))
    l -->|1.0| o((/o/))
    o -->|1.0| o
    
    h -.-> O1[O1]
    e -.-> O2[O2]
    l -.-> O3[O3]
    o -.-> O4[O4]
```
*Note: Transition probabilities shown are simplified for the 'hello' example.*

In [ ]:
# 3. Define the HMM Components

# States (Phonemes)
states = ['h', 'e', 'l', 'o']

# Observations (Acoustic Features)
observations = ['O1', 'O2', 'O3', 'O4']

# Transition Probability Matrix A
# A[i][j] = P(state_j | state_i)
A = {
    'h': {'h': 0.0, 'e': 1.0, 'l': 0.0, 'o': 0.0},
    'e': {'h': 0.0, 'e': 0.0, 'l': 1.0, 'o': 0.0},
    'l': {'h': 0.0, 'e': 0.0, 'l': 0.0, 'o': 1.0},
    'o': {'h': 0.0, 'e': 0.0, 'l': 0.0, 'o': 1.0}
}

# Emission Probability Matrix B
# B[i][k] = P(observation_k | state_i)
B = {
    'h': {'O1': 1.0, 'O2': 0.0, 'O3': 0.0, 'O4': 0.0},
    'e': {'O1': 0.0, 'O2': 1.0, 'O3': 0.0, 'O4': 0.0},
    'l': {'O1': 0.0, 'O2': 0.0, 'O3': 1.0, 'O4': 0.0},
    'o': {'O1': 0.0, 'O2': 0.0, 'O3': 0.0, 'O4': 1.0}
}

# Initial Probabilities pi
pi = {'h': 1.0, 'e': 0.0, 'l': 0.0, 'o': 0.0}

# Observation Sequence to Decode
obs_sequence = ['O1', 'O2', 'O3', 'O4']

print("HMM Components Defined.")

## 4. Explain the Viterbi Algorithm

The Viterbi algorithm finds the most likely sequence of hidden states $Q$ given the observations $O$.

### Steps:

1.  **Initialization**: 
    - Calculate the probability of starting in each state $i$ and generating the first observation $O_1$.
    - $\delta_1(i) = \pi_i \cdot B_i(O_1)$
    - $\psi_1(i) = 0$ (Backpointer)

2.  **Recursion** (for $t=2$ to $T$):
    - For each state $j$, find the best previous state $i$ that maximizes the probability of transitioning to $j$ and emitting $O_t$.
    - $\delta_t(j) = \max_{i} [\delta_{t-1}(i) \cdot A_{ij}] \cdot B_j(O_t)$
    - $\psi_t(j) = \arg\max_{i} [\delta_{t-1}(i) \cdot A_{ij}]$

3.  **Termination**:
    - Find the final best path probability and the last state.
    - $P^* = \max_{i} [\delta_T(i)]$
    - $q_T^* = \arg\max_{i} [\delta_T(i)]$

4.  **Backtracking**:
    - Trace back from the last state to the first to reconstruct the path.
    - $q_t^* = \psi_{t+1}(q_{t+1}^*)$

### Why Log Probabilities?
Probabilities in HMMs can become very small (vanishing gradients problem) as we multiply them over long sequences. Using **log probabilities** turns multiplication into addition ($log(a \cdot b) = log(a) + log(b)$), which is numerically stable.

### Flowchart

```mermaid
flowchart TD
    Start([Start]) --> Init[Initialization: t=1]
    Init --> Recurse{t <= T?}
    Recurse -- Yes --> Calc[Calculate max prob for each state]
    Calc --> Store[Store backpointers]
    Store --> Inc[t = t + 1]
    Inc --> Recurse
    Recurse -- No --> Term[Termination: Find max prob at T]
    Term --> Back[Backtrack to find path]
    Back --> End([End])
```

In [ ]:
# 5. Implement the Viterbi Algorithm

import numpy as np
import pandas as pd

def viterbi(states, observations, A, B, pi, obs_seq):
    T = len(obs_seq)
    N = len(states)
    
    # Initialize DP table (delta) and backpointer table (psi)
    # Rows: States, Cols: Time steps
    delta = pd.DataFrame(0.0, index=states, columns=range(T))
    psi = pd.DataFrame("", index=states, columns=range(T))
    
    # 1. Initialization
    first_obs = obs_seq[0]
    for s in states:
        delta.loc[s, 0] = pi[s] * B[s][first_obs]
        psi.loc[s, 0] = "Start"
        
    # 2. Recursion
    for t in range(1, T):
        obs = obs_seq[t]
        for s_curr in states:
            # Calculate prob of arriving at s_curr from each s_prev
            probs = {}
            for s_prev in states:
                # P(prev) * Transition(prev->curr) * Emission(curr emits obs)
                p = delta.loc[s_prev, t-1] * A[s_prev][s_curr] * B[s_curr][obs]
                probs[s_prev] = p
            
            # Find max prob and the state that gave it
            max_prob = max(probs.values())
            best_prev_state = max(probs, key=probs.get)
            
            delta.loc[s_curr, t] = max_prob
            psi.loc[s_curr, t] = best_prev_state
            
    # 3. Termination
    # Find the state with the highest probability at the last time step
    last_col = delta.iloc[:, T-1]
    best_path_prob = last_col.max()
    last_state = last_col.idxmax()
    
    # 4. Backtracking
    best_path = [""] * T
    best_path[T-1] = last_state
    
    for t in range(T-2, -1, -1):
        best_path[t] = psi.loc[best_path[t+1], t+1]
        
    return best_path, best_path_prob, delta

print("Viterbi Function Defined.")

In [ ]:
# 6. Run the Algorithm and Show Output

best_sequence, probability, dp_table = viterbi(states, observations, A, B, pi, obs_sequence)

print("Dynamic Programming Table (Delta):")
print(dp_table)
print("\n" + "="*30)
print(f"Most likely sequence: {' -> '.join(best_sequence)}")
print(f"Probability: {probability}")
print("="*30)

In [ ]:
# 7. Plot Probability Evolution
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.heatmap(dp_table, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("Viterbi Path Probability Evolution (Heatmap)")
plt.xlabel("Time Step (Observations)")
plt.ylabel("States (Phonemes)")
plt.xticks(ticks=[0.5, 1.5, 2.5, 3.5], labels=obs_sequence)
plt.show()

# Line plot visualization
plt.figure(figsize=(10, 6))
for state in states:
    plt.plot(range(len(obs_sequence)), dp_table.loc[state], marker='o', label=f'State {state}')

plt.title("State Probability Progression")
plt.xlabel("Time Step")
plt.ylabel("Probability")
plt.xticks(range(len(obs_sequence)), obs_sequence)
plt.legend()
plt.grid(True)
plt.show()

## 8. Final Inference

### Conclusion
The Viterbi algorithm successfully decoded the observation sequence `['O1', 'O2', 'O3', 'O4']` into the phoneme sequence `h -> e -> l -> o` with a probability of **1.0**.

### Why this path?
1.  **Initialization**: The initial probability vector $\pi$ strongly favored starting at 'h' (prob 1.0).
2.  **Transitions**: The transition matrix $A$ was set up deterministically for this example (e.g., 'h' only goes to 'e').
3.  **Emissions**: The emission matrix $B$ perfectly aligned observations with states (e.g., 'h' emits 'O1' with prob 1.0).

In a real-world scenario, these probabilities would not be 0 or 1, but learned from data. The Viterbi algorithm is robust because it considers the *entire* sequence context to resolve ambiguities, rather than just picking the best state at each individual moment.